In [5]:
""" A simple helloworld example
Different workflows are shown here.
"""
from tensorflow import keras
from tensorflow.keras import layers

from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.tuners import RandomSearch #conda install -c conda-forge keras-tuner



In [11]:
from tensorflow.keras.datasets import mnist
# (x_train, y_train), (x_test, y_test) = mnist.load_data(path='./mnist.npz')
# path.exists('datasets/mnist_x.npy'):
(x, y), (val_x, val_y) = mnist.load_data(path='d:\Projects\AI\POC\MPS\mnist.npz')
x = x.astype("float32") / 255.0
val_x = val_x.astype("float32") / 255.0

x = x[:10000]
y = y[:10000]


"""Basic case:
- We define a `build_model` function
- It returns a compiled model
- It uses hyperparameters defined on the fly
"""

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz: None -- [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。

In [ ]:


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.Int("num_layers", 2, 20)):
        model.add(
            layers.Dense(
                units=hp.Int("units_" + str(i), 32, 512, 32), activation="relu"
            )
        )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model



In [ ]:

tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=3,
    directory="test_dir",
    project_name="case1",
)

tuner.search_space_summary()

tuner.search(x=x, y=y, epochs=3, validation_data=(val_x, val_y))

tuner.results_summary()




In [ ]:
# """Case #2:
# - We override the loss and metrics
# """

tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    loss=keras.losses.SparseCategoricalCrossentropy(name="my_loss"),
    metrics=["accuracy", "mse"],
    max_trials=5,
    directory="test_dir",
    project_name="case2",
)

tuner.search(x, y, epochs=5, validation_data=(val_x, val_y))


# """Case #3:
# - We define a HyperModel subclass
# """




In [ ]:
class MyHyperModel(HyperModel):
    def __init__(self, img_size, classes):
        self.img_size = img_size
        self.classes = classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten(input_shape=self.img_size))
        for i in range(hp.Int("num_layers", 2, 20)):
            model.add(
                layers.Dense(
                    units=hp.Int("units_" + str(i), 32, 512, 32), activation="relu"
                )
            )
        model.add(layers.Dense(self.classes, activation="softmax"))
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
            ),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model




In [ ]:
tuner = RandomSearch(
    MyHyperModel(img_size=(28, 28), classes=10),
    objective="val_accuracy",
    max_trials=5,
    directory="test_dir",
    project_name="case3",
)

tuner.search(x, y=y, epochs=5, validation_data=(val_x, val_y))





In [ ]:
# """Case #4:
# - We restrict the search space
# - This means that default values are being used for params that are left out
# """

hp = HyperParameters()
hp.Choice("learning_rate", [1e-1, 1e-3])

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=False,
    objective="val_accuracy",
    directory="test_dir",
    project_name="case4",
)

tuner.search(x=x, y=y, epochs=5, validation_data=(val_x, val_y))

In [ ]:
# """Case #5:
# - We override specific parameters with fixed values that aren't the default
# """

hp = HyperParameters()
hp.Fixed("learning_rate", 0.1)

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=True,
    objective="val_accuracy",
    directory="test_dir",
    project_name="case5",
)

tuner.search(x=x, y=y, epochs=5, validation_data=(val_x, val_y))



In [ ]:

# """Case #6:
# - We reparameterize the search space
# - This means that we override the distribution of specific hyperparameters
# """

hp = HyperParameters()
hp.Choice("learning_rate", [1e-1, 1e-3])

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=True,
    objective="val_accuracy",
    directory="test_dir",
    project_name="case6",
)

tuner.search(x=x, y=y, epochs=5, validation_data=(val_x, val_y))




In [ ]:
# """Case #7:
# - We predefine the search space
# - No unregistered parameters are allowed in `build`
# """

hp = HyperParameters()
hp.Choice("learning_rate", [1e-1, 1e-3])
hp.Int("num_layers", 2, 20)


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.get("num_layers")):
        model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.get("learning_rate")),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    allow_new_entries=False,
    objective="val_accuracy",
    directory="test_dir",
    project_name="case7",
)

tuner.search(x=x, y=y, epochs=5, validation_data=(val_x, val_y))







In [ ]:
# """Case #8:
# - Similar to Base Case.
# - However, specify conditions on units so that the summary show
# - only relevant hyperparameters.
# """


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    min_layers = 2
    max_layers = 5
    for i in range(hp.Int("num_layers", min_layers, max_layers)):
        with hp.conditional_scope("num_layers", list(range(i + 1, max_layers + 1))):
            model.add(
                layers.Dense(
                    units=hp.Int("units_" + str(i), 32, 256, 32), activation="relu"
                )
            )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=3,
    directory="test_dir",
)

tuner.search_space_summary()

tuner.search(x=x, y=y, epochs=3, validation_data=(val_x, val_y))

tuner.results_summary()

In [ ]:
# """Case #9:
# - Similar to Case #8, but use parent_name, parent_value keywords pair for
# - conditional scope Using keywords for conditional scope does not
# - support nested conditions.
# """


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    min_layers = 2
    max_layers = 5
    for i in range(hp.Int("num_layers", min_layers, max_layers)):
        model.add(
            layers.Dense(
                units=hp.Int(
                    "units_" + str(i),
                    32,
                    256,
                    32,
                    parent_name="num_layers",
                    parent_values=list(range(i + 1, max_layers + 1)),
                ),
                activation="relu",
            )
        )
        model.add(layers.Dense(10, activation="softmax"))
        model.compile(
            optimizer=keras.optimizers.Adam(1e-4),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model


tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=3,
    directory="test_dir",
)

tuner.search_space_summary()

tuner.search(x=x, y=y, epochs=3, validation_data=(val_x, val_y))

tuner.results_summary()